In [1715]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil

# Mueve todos los fichero del directorio de trabajo a otro de archivo

In [1716]:
source_dir = './BOEs'
target_dir = './BOEs_Anteriores'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    #shutil.move(os.path.join(source_dir, file_name), target_dir)
    # Evita que de error si el fichero que se mueve ya existe en dir destino
    try:
        os.remove(os.path.join(target_dir, file_name))
        shutil.move(os.path.join(source_dir, file_name), target_dir)
    except OSError:
        shutil.move(os.path.join(source_dir, file_name), target_dir)

# Recoge resumen diario del BOE de hoy

In [1717]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210212
dia = 12
mes = 02
año = 2021
12/02/2021


In [1718]:
URL_XML_resumen =  "https://www.boe.es/diario_boe/xml.php?id=BOE-S-" + str(hoy)

In [1719]:
URL_XML_resumen

'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20210212'

In [1720]:
url = URL_XML_resumen
r = requests.get(url)

In [1721]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

save_html(r.content, './BOEs/Resumen-BOE-' + hoy + '.xml')

In [1722]:
resumen = etree.parse('./BOEs/Resumen-BOE-' + hoy + '.xml')

In [1723]:
raiz=resumen.getroot()
raiz.tag
raiz_sumario = raiz

In [1724]:
seccion = raiz.findall("sumario/diario/seccion")
seccion

[]

In [1725]:
for seccion in raiz.xpath('//seccion'):
    nombre_seccion = seccion.xpath('@nombre')
    print(nombre_seccion)

['I. Disposiciones generales']
['II. Autoridades y personal. - A. Nombramientos, situaciones e incidencias']
['II. Autoridades y personal. - B. Oposiciones y concursos']
['III. Otras disposiciones']
['IV. Administración de Justicia']
['V. Anuncios. - A. Contratación del Sector Público']
['V. Anuncios. - B. Otros anuncios oficiales']


In [1726]:
secciones = raiz.findall("sumario/diario/seccion")
for seccion in secciones:
    print(seccion.text)

In [1727]:
tabla_resumen = pd.DataFrame()

In [1728]:
for seccion in raiz.xpath('//seccion[contains(@nombre, "I. Disposiciones generales")]'):
    # el = doc.xpath("//div[contains(@class, 'channel') and not(contains(@class, 'disabled'))]")
    nombre_seccion = seccion.xpath('@nombre')

    for departamento in seccion:
        nombre_departamento = departamento.xpath('@nombre') 

        for epigrafe in departamento:
            nombre_epigrafe = epigrafe.xpath('@nombre')

            for item in epigrafe:
                item_id = item.xpath('@id')
                item_name = item.xpath('.//titulo/text()') 

                item_urlXml = "https://www.boe.es" + str(item.xpath('.//urlXml/text()'))[2:-2]

                tabla_resumen = tabla_resumen.append({'Seccion': nombre_seccion, 
                                                      'Departamento': nombre_departamento, 
                                                      'Epigrafe' : nombre_epigrafe,
                                                      'Item_id': item_id, 
                                                      'Item_Nombre' : item_name, 
                                                      'Item_URL_XML' : item_urlXml},
                                                      ignore_index=True)


In [1729]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[MINISTERIO DE HACIENDA],[Fondo Europeo de Desarrollo Regional. Program...,"[Orden HAC/114/2021, de 5 de febrero, por la q...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2045],[I. Disposiciones generales]
1,[MINISTERIO DE CONSUMO],[Material sanitario],"[Orden CSM/115/2021, de 11 de febrero, por la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2046],[I. Disposiciones generales]
2,[COMUNIDAD AUTÓNOMA DEL PAÍS VASCO],[Circulación. Medidas especiales],"[Resolución de 3 de febrero de 2021, de la Dir...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-2047],[I. Disposiciones generales]


# Descarga ficheros XML asociados

In [1730]:
for item_URL in tabla_resumen['Item_URL_XML']:
    r = requests.get(item_URL)
    #f = './BOEs/' + item_URL[-16:] + '.xml'
    ### Separa el número del BOE del resto de la cadena y aplica expresión REGEX 
    print(item_URL.split('='))
    print(re.match('BOE\-A\-[0-9]+\-[0-9]+',item_URL.split('=')[1]))
    filename = re.match('BOE\-A\-[0-9]+\-[0-9]+',item_URL.split('=')[1]).group()
    f = './BOEs/' + filename + '.xml'
    save_html(r.content, f)

['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2045']
<re.Match object; span=(0, 15), match='BOE-A-2021-2045'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2046']
<re.Match object; span=(0, 15), match='BOE-A-2021-2046'>
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-2047']
<re.Match object; span=(0, 15), match='BOE-A-2021-2047'>


# Importa de nuevo los XML generados

In [1731]:

import glob
BOEs = glob.glob('./BOEs/BOE*.xml')
print (BOEs)

['./BOEs/BOE-A-2021-2045.xml', './BOEs/BOE-A-2021-2047.xml', './BOEs/BOE-A-2021-2046.xml']


# Genera DF con datos Análisis de cada XML

In [1732]:
tabla_analisis = pd.DataFrame()

for BOE in BOEs:
    #print (BOE)
    BOE_XML = etree.parse(BOE)
    raiz=BOE_XML.getroot()

    materias = [materia.text for materia in raiz.findall('analisis/materias/materia')]
    alertas = [alerta.text for alerta in raiz.findall('analisis/alertas/alerta')]
    
    referencias = [str(referencia.xpath('@referencia')).replace("['",'').replace("']",'') for referencia in raiz.findall('analisis/referencias/anteriores/anterior')]
    referencias_palabras = [referencia.text for referencia in raiz.findall('analisis/referencias/anteriores/anterior/palabra')]
    referencias_texto = [referencia.text for referencia in raiz.findall('analisis/referencias/anteriores/anterior/texto')]

    Item_Id = raiz.xpath(".//identificador/text()")
    Item_name = raiz.xpath('.//titulo/text()') 

    Fecha_Publicacion = datetime.strptime(str(raiz.xpath('.//fecha_publicacion/text()'))[2:-2],"%Y%m%d")
    #Fecha_Publicacion = str(raiz.xpath('.//fecha_publicacion/text()'))[2:-2]
 
    tabla_analisis = tabla_analisis.append({'Item_id': Item_Id,
                                            'Item_Name' : Item_name,
                                            'Fecha_publicacion' : Fecha_Publicacion,
                                            'Materias' : materias,
                                            'Alertas' : alertas,
                                            'Referencias' : referencias,
                                            'Referencias_palabra' : referencias_palabras,
                                            'Referencias_texto' : referencias_texto},
                                            ignore_index=True)


In [1733]:
tabla_analisis.sort_values('Item_id')

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto
0,"[Sistema financiero, Unión Europea]",2021-02-12,"[Orden HAC/114/2021, de 5 de febrero, por la q...",[BOE-A-2021-2045],"[Ayudas, Desarrollo regional, Financiación com...","[BOE-A-2016-12485, BOE-A-2003-20977, DOUE-L-20...","[MODIFICA, DE CONFORMIDAD con, CITA, CITA, CITA]","[las normas 1, 3, 6, 10, 12 a 16, SE AÑADE las..."
2,"[Comercio, Consumidores y usuarios, Sanidad, T...",2021-02-12,"[Orden CSM/115/2021, de 11 de febrero, por la ...",[BOE-A-2021-2046],"[Comercialización, Entidades de acreditación, ...","[BOE-A-2020-4761, BOE-A-2020-4525]","[DE CONFORMIDAD con, EN RELACIÓN con]","[el art. 3 del Real Decreto 495/2020, de 28 de..."
1,[Transportes y tráfico],2021-02-12,"[Resolución de 3 de febrero de 2021, de la Dir...",[BOE-A-2021-2047],"[Carreteras, Circulación vial, Deporte, Mercan...","[BOE-A-2015-11722, BOE-A-2003-23514, BOE-A-201...","[DE CONFORMIDAD con, DE CONFORMIDAD con, EN RE...","[el art. 18 de la Ley sobre Tráfico, Circulaci..."


In [1734]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
tabla_analisis['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
tabla_analisis['Tags'] = [[] for i in range(len(tabla_analisis))]
tabla_analisis['Match_ASECORP_BBDD'] = [[] for i in range(len(tabla_analisis))]


In [1735]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
pattern = ['Ley [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+','Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Circular [0-9]+\/[0-9]+']

In [1736]:
# Consolida las columnas Referencias_palabra y Referencias_texto en una única frase
for i, row in tabla_analisis.iterrows():
    for item_list in range(len(row['Referencias'])): 
        row['Referencias_completas'].append(row['Referencias_palabra'][item_list] + ' ' + row['Referencias_texto'][item_list])
        tabla_analisis['Tags'][i] = re.findall('|'.join(pattern), str(row['Referencias_texto']), flags=re.IGNORECASE)
    print(row['Referencias_completas'])

['MODIFICA las normas 1, 3, 6, 10, 12 a 16, SE AÑADE las  normas 10 bis y 14 bis y SE SUPRIME la norma 19 de la Orden HFP/1979/2016, de 29 de diciembre', 'DE CONFORMIDAD con la Ley 38/2003, de 17 de noviembre', 'CITA Reglamento (UE) 2020/558, de 23 de abril', 'CITA Reglamento (UE) 2020/460, de 30 de marzo', 'CITA Reglamento (UE) 2018/1046, de 18 de julio']
['DE CONFORMIDAD con el art. 18 de la Ley sobre Tráfico, Circulación de Vehículos a Motor y Seguridad Vial, texto refundido aprobado por Real Decreto Legislativo 6/2015, de 30 de octubre', 'DE CONFORMIDAD con los arts. 37 y 39 del Reglamento aprobado por Real Decreto 1428/2003, de 21 de noviembre', 'EN RELACIÓN sobre la Resolución de 20 de diciembre de 2019']
['DE CONFORMIDAD con el art. 3 del Real Decreto 495/2020, de 28 de abril', 'EN RELACIÓN con la Orden SND/354/2020, de 19 de abril']


In [1737]:
tabla_analisis

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto,Referencias_completas,Tags,Match_ASECORP_BBDD
0,"[Sistema financiero, Unión Europea]",2021-02-12,"[Orden HAC/114/2021, de 5 de febrero, por la q...",[BOE-A-2021-2045],"[Ayudas, Desarrollo regional, Financiación com...","[BOE-A-2016-12485, BOE-A-2003-20977, DOUE-L-20...","[MODIFICA, DE CONFORMIDAD con, CITA, CITA, CITA]","[las normas 1, 3, 6, 10, 12 a 16, SE AÑADE las...","[MODIFICA las normas 1, 3, 6, 10, 12 a 16, SE ...","[Orden HFP/1979/2016, Ley 38/2003, Reglamento ...",[]
1,[Transportes y tráfico],2021-02-12,"[Resolución de 3 de febrero de 2021, de la Dir...",[BOE-A-2021-2047],"[Carreteras, Circulación vial, Deporte, Mercan...","[BOE-A-2015-11722, BOE-A-2003-23514, BOE-A-201...","[DE CONFORMIDAD con, DE CONFORMIDAD con, EN RE...","[el art. 18 de la Ley sobre Tráfico, Circulaci...",[DE CONFORMIDAD con el art. 18 de la Ley sobre...,"[Real Decreto Legislativo 6/2015, Real Decreto...",[]
2,"[Comercio, Consumidores y usuarios, Sanidad, T...",2021-02-12,"[Orden CSM/115/2021, de 11 de febrero, por la ...",[BOE-A-2021-2046],"[Comercialización, Entidades de acreditación, ...","[BOE-A-2020-4761, BOE-A-2020-4525]","[DE CONFORMIDAD con, EN RELACIÓN con]","[el art. 3 del Real Decreto 495/2020, de 28 de...",[DE CONFORMIDAD con el art. 3 del Real Decreto...,"[Real Decreto 495/2020, Orden SND/354/2020]",[]


In [1738]:
texto = ''
for refs in tabla_analisis['Referencias_completas']:
    for ref in refs:
        texto = texto + ref + ' '
        print(ref)

MODIFICA las normas 1, 3, 6, 10, 12 a 16, SE AÑADE las  normas 10 bis y 14 bis y SE SUPRIME la norma 19 de la Orden HFP/1979/2016, de 29 de diciembre
DE CONFORMIDAD con la Ley 38/2003, de 17 de noviembre
CITA Reglamento (UE) 2020/558, de 23 de abril
CITA Reglamento (UE) 2020/460, de 30 de marzo
CITA Reglamento (UE) 2018/1046, de 18 de julio
DE CONFORMIDAD con el art. 18 de la Ley sobre Tráfico, Circulación de Vehículos a Motor y Seguridad Vial, texto refundido aprobado por Real Decreto Legislativo 6/2015, de 30 de octubre
DE CONFORMIDAD con los arts. 37 y 39 del Reglamento aprobado por Real Decreto 1428/2003, de 21 de noviembre
EN RELACIÓN sobre la Resolución de 20 de diciembre de 2019
DE CONFORMIDAD con el art. 3 del Real Decreto 495/2020, de 28 de abril
EN RELACIÓN con la Orden SND/354/2020, de 19 de abril


In [1739]:
texto = ''
for i, row in tabla_analisis.iterrows():
    #unique_id = i
    print('\n' + str(row['Item_id']) + str(row['Item_Name']) +'\n')

    antecedente = 1
    for ref in row['Referencias_completas']:
        texto = texto + ref + ' '
        print('\t' + 'Antecedente ' + str(antecedente) + ': ' + ref + ' ' + str(re.findall('|'.join(pattern), ref, flags=re.IGNORECASE)))
        antecedente += 1


['BOE-A-2021-2045']['Orden HAC/114/2021, de 5 de febrero, por la que se modifica la Orden HFP/1979/2016, de 29 de diciembre, por la que se aprueban las normas sobre los gastos subvencionables de los Programas Operativos del Fondo Europeo de Desarrollo Regional para el período 2014-2020.']

	Antecedente 1: MODIFICA las normas 1, 3, 6, 10, 12 a 16, SE AÑADE las  normas 10 bis y 14 bis y SE SUPRIME la norma 19 de la Orden HFP/1979/2016, de 29 de diciembre ['Orden HFP/1979/2016']
	Antecedente 2: DE CONFORMIDAD con la Ley 38/2003, de 17 de noviembre ['Ley 38/2003']
	Antecedente 3: CITA Reglamento (UE) 2020/558, de 23 de abril ['Reglamento (UE) 2020/558']
	Antecedente 4: CITA Reglamento (UE) 2020/460, de 30 de marzo ['Reglamento (UE) 2020/460']
	Antecedente 5: CITA Reglamento (UE) 2018/1046, de 18 de julio ['Reglamento (UE) 2018/1046']

['BOE-A-2021-2047']['Resolución de 3 de febrero de 2021, de la Dirección de Tráfico, del Departamento de Seguridad, por la que se establecen medidas especia

In [1740]:
# Aplica expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
regex_result = re.findall('|'.join(pattern), texto, flags=re.IGNORECASE)

In [1741]:
print(regex_result)

['Orden HFP/1979/2016', 'Ley 38/2003', 'Reglamento (UE) 2020/558', 'Reglamento (UE) 2020/460', 'Reglamento (UE) 2018/1046', 'Real Decreto Legislativo 6/2015', 'Real Decreto 1428/2003', 'Resolución de 20 de diciembre de 2019', 'Real Decreto 495/2020', 'Orden SND/354/2020']


In [1742]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [1743]:
boletin_flat_list

['Resolución de 20 de diciembre de 2019',
 'Reglamento (UE) 2018/1046',
 'Real Decreto Legislativo 6/2015',
 'Reglamento (UE) 2020/460',
 'Ley 38/2003',
 'Real Decreto 495/2020',
 'Real Decreto 1428/2003',
 'Orden SND/354/2020',
 'Reglamento (UE) 2020/558',
 'Orden HFP/1979/2016']

## Importa BBDD ASECORP

In [1744]:
ASECORP_BBDD = pd.read_csv('./ASECORP/ExportNormes_20210126.csv', delimiter=';')

In [1745]:
ASECORP_BBDD

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
0,RES SLT/2313/2020 CAT,2313,"Resolución SLT/2313/2020, de 24 de septiembre,...",CAT,Cataluña,NaN,Prevención,[2]
1,(zzz) RES 12/4/2007 AST,12/4/2007,"Resolución de 12 abril de 2007, de la Consejer...",AST,Asturias,NaN,M. Ambiente,[1]
2,A CORUÑA 10/7/2015,10/7/2015,Modificación de la Ordenanza de vertidos y ser...,GAL,Galicia,A Coruña,M. Ambiente,[1]
3,A CORUÑA 11/8/2014 (1),11/8/2014,Ordenanza municipal reguladora de los procedim...,GAL,Galicia,A Coruña,M. Ambiente,[1]
4,A CORUÑA 11/8/2014 (2),11/8/2014,Ordenanza de protección contra la contaminació...,GAL,Galicia,A Coruña,M. Ambiente,[1]
...,...,...,...,...,...,...,...,...
19152,ZUMAIA 1/3/2001,1/3/2001,Ordenanza reguladora de la gestión del ciclo i...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19153,ZUMAIA 12/11/1999,12/11/1999,Normativa complementaria de las Normas Subsidi...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19154,ZUMAIA 15/6/2012,15/6/2012,Modificación de la Ordenanza municipal de la g...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19155,ZUMAIA 6/2/2004,6/2/2004,Modificación de la Normativa complementaria de...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]


In [1746]:
ASECORP_BBDD_BOE = ASECORP_BBDD.loc[ASECORP_BBDD['Ambito'] == 'España']

In [1747]:
ASECORP_BBDD_BOE

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
23,ACU 1/2020 ESP,1,Acuerdo Multilateral RID 1/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
35,ACU 11/6/2007 ESP,11/6/2007,Entrada en vigor del Acuerdo entre el Reino de...,ESP,España,NaN,M. Ambiente,[1]
57,ACU 17/10/2006 ESP,17/10/2006,Aplicación provisional del Acuerdo entre el Re...,ESP,España,NaN,M. Ambiente,[1]
71,ACU 19/11/2018 ESP,19/11/2018,Acuerdo de la Junta de Gobierno de la Confeder...,ESP,España,NaN,M. Ambiente,[1]
82,ACU 2/2020 ESP,2,Acuerdo Multilateral RID 2/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
...,...,...,...,...,...,...,...,...
18496,STC 86/2019 ESP,86,"Sentencia 86/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18497,STC 87/2019 ESP,87,"Sentencia 87/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18498,STC 88/2018 ESP,88,"Sentencia 88/2018, de 19 de julio de 2018. Con...",ESP,España,NaN,M. Ambiente / Reg. Industrial,[1][3]
18499,STC 9/3/2010 ESP,9/3/2010,"Sentencia de 9 de marzo de 2010, de la Sala Te...",ESP,España,NaN,M. Ambiente,[1]


In [1748]:
# Crea nueva columna vacía de tipo lista en ASECORP_BBDD
ASECORP_BBDD_BOE['Tags'] = [[] for i in range(len(ASECORP_BBDD_BOE))]

In [1749]:
titulo = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    titulo.append(str(row['Titulo']) + str(re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)))
    ASECORP_BBDD_BOE['Tags'][i] = re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)

In [1750]:
ASECORP_BBDD_BOE[1200:1250]

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma,Tags
11629,ORD PRE/2827/2009 ESP,2827,"Orden PRE/2827/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/2827/2009, Real Decreto 1370/2006]"
11630,ORD PRE/2843/2009 ESP,2843,"Orden PRE/2843/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2843/2009, Real Decreto 2163/1994]"
11631,ORD PRE/2851/2010 ESP,2851,"Orden PRE/2851/2010, de 4 de noviembre, por la...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2851/2010, Real Decreto 2163/1994]"
11632,ORD PRE/2871/2011 ESP,2871,"Orden PRE/2871/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2871/2011, Real Decreto 2163/1994]"
11633,ORD PRE/2872/2011 ESP,2872,"Orden PRE/2872/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2872/2011, Real Decreto 2163/1994]"
11634,ORD PRE/29/2004 ESP,29,"Orden PRE/29/2004, de 15 de enero, por la que ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/29/2004, Real Decreto 837/2002]"
11635,ORD PRE/2922/2005 ESP,2922,"Orden PRE/2922/2005, de 19 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2922/2005, Orden de 8 de marzo de 1..."
11636,ORD PRE/2957/2008 ESP,2957,"Orden PRE/2957/2008, de 10 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2957/2008, Real Decreto 2163/1994]"
11637,ORD PRE/3/2006 ESP,3,"Orden PRE/3/2006, de 12 de enero, por la que s...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3/2006, Real Decreto 255/2003]"
11638,ORD PRE/3159/2004 ESP,3159,"Orden PRE/3159/2004, de 28 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3159/2004, Real Decreto 1406/1989]"


In [1751]:
# titulo

In [1752]:
len(titulo)

2959

In [1753]:
## Busca patrones regex definidos en columna títulos del DF 
boletin = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    boletin.append(str(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)))

In [1754]:
boletin[0:25]

['[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral M-150']",
 "['Acuerdo Multilateral M-159']",
 "['Acuerdo multilateral M-168']",
 "['Acuerdo Multilateral M-170']",
 "['Acuerdo Multilateral M-171']",
 "['Acuerdo Multilateral M-173']",
 "['Acuerdo Multilateral M-175']",
 '[]',
 "['Acuerdo Multilateral M-178']",
 "['Acuerdo Multilateral M-198']"]

In [1755]:
## Para cada fila de la BBDD recoge la expresión REGEX encontrada y si no existe no la incluye 
## en la lista resultante llamada 'boletin'. Además si no detecta expresión la cuenta como vacía
## en variable 'n_vacios', y la añade a la lista 'vacios' para inspeccionar posteriormente 
boletin = []
vacios = []
n_vacios = 0
for i, row in ASECORP_BBDD_BOE.iterrows():
    regex_result = re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)
    if len(regex_result) != 0:
        boletin.append(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE))
    else:
        n_vacios += 1
        vacios.append(row['Titulo'])

In [1756]:
n_vacios

276

In [1757]:
vacios[0:25]

['Acuerdo Multilateral RID 1/2020 en virtud de la Sección 1.5.1 del Reglamento del Transporte Internacional de Mercancías Peligrosas por Ferrocarril (RID), relativo a los certificados de consejeros de seguridad de conformidad con el punto 1.8.3.7 del RID, hecho en Madrid el 25 de marzo de 2020',
 'Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Aplicación provisional del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Acuerdo de la Junta de Gobierno de la Confederación Hidrográfica del Segura celebrada el día 30 de octubre de 2018 sobre aplicación de indisponibilidad o reducción de dotaciones de acuerdo al Plan Especial de Sequía y al Real Decreto que la regula',
 'Acuerdo Multilateral RID 2/2020 en virtud de la Sección 1.5.1 del Reglamento del Transporte Internacional de Merc

In [1758]:
boletin[0:25]

[['Acuerdo Multilateral M-150'],
 ['Acuerdo Multilateral M-159'],
 ['Acuerdo multilateral M-168'],
 ['Acuerdo Multilateral M-170'],
 ['Acuerdo Multilateral M-171'],
 ['Acuerdo Multilateral M-173'],
 ['Acuerdo Multilateral M-175'],
 ['Acuerdo Multilateral M-178'],
 ['Acuerdo Multilateral M-198'],
 ['Acuerdo Multilateral M-214'],
 ['Acuerdo Multilateral M-218'],
 ['Acuerdo Multilateral M-226'],
 ['Acuerdo Multilateral M-228'],
 ['Acuerdo Multilateral M-231'],
 ['Acuerdo Multilateral M-236'],
 ['Acuerdo Multilateral M-245'],
 ['Acuerdo Multilateral M-254'],
 ['Acuerdo Multilateral M-259'],
 ['Acuerdo Multilateral M-265'],
 ['Acuerdo Multilateral M-268'],
 ['Acuerdo Multilateral M-271'],
 ['Acuerdo Multilateral M-273'],
 ['Acuerdo Multilateral M-276'],
 ['Acuerdo Multilateral M-291'],
 ['Acuerdo Multilateral M-292']]

In [1759]:
# Flatten list of lists
boletin_ASECORP_flat_list = [item for sublist in boletin for item in sublist]

In [1760]:
boletin_ASECORP_flat_list[0:25]

['Acuerdo Multilateral M-150',
 'Acuerdo Multilateral M-159',
 'Acuerdo multilateral M-168',
 'Acuerdo Multilateral M-170',
 'Acuerdo Multilateral M-171',
 'Acuerdo Multilateral M-173',
 'Acuerdo Multilateral M-175',
 'Acuerdo Multilateral M-178',
 'Acuerdo Multilateral M-198',
 'Acuerdo Multilateral M-214',
 'Acuerdo Multilateral M-218',
 'Acuerdo Multilateral M-226',
 'Acuerdo Multilateral M-228',
 'Acuerdo Multilateral M-231',
 'Acuerdo Multilateral M-236',
 'Acuerdo Multilateral M-245',
 'Acuerdo Multilateral M-254',
 'Acuerdo Multilateral M-259',
 'Acuerdo Multilateral M-265',
 'Acuerdo Multilateral M-268',
 'Acuerdo Multilateral M-271',
 'Acuerdo Multilateral M-273',
 'Acuerdo Multilateral M-276',
 'Acuerdo Multilateral M-291',
 'Acuerdo Multilateral M-292']

In [1761]:
## Elimina duplicados
boletin_ASECORP_flat_list = list(set(boletin_ASECORP_flat_list))

In [1762]:
boletin_ASECORP_flat_list[0:25]

['Resolución de 9 de septiembre de 2013',
 'Orden SND/440/2020',
 'Nota de servicio 1/2014',
 'Nota de servicio 1/2010',
 'Resolución de 31 de octubre de 2012',
 'Real Decreto 635/2013',
 'Orden ITC/3163/2009',
 'Orden TMA/400/2020',
 'Ley 12/2007',
 'Sentencia de 24 de enero de 2011',
 'Ley 5/2014',
 'Real Decreto 1528/2012',
 'Sentencia 61/2015',
 'Real Decreto 45/1996',
 'Orden de 13 de diciembre de 2000',
 'Ley 11/2012',
 'Real Decreto 1338/2011',
 'Resolución de 8 de febrero de 2018',
 'Real Decreto 134/2010',
 'Orden PRE/3539/2008',
 'Resolución de 20 de noviembre de 2015',
 'Real Decreto 1315/2005',
 'Resolución de 25 de abril de 2005',
 'Real Decreto 895/2013',
 'Real Decreto 1837/2008']

In [1763]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

{'Real Decreto 1428/2003', 'Real Decreto Legislativo 6/2015'}

In [1764]:
#tabla_analisis['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in tabla_analisis['Tags']:
    for row_comparing in ASECORP_BBDD_BOE['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

{'Real Decreto 1428/2003'}
{'Real Decreto 1428/2003'}
{'Real Decreto 1428/2003'}
{'Real Decreto 1428/2003'}
{'Real Decreto Legislativo 6/2015'}


In [1765]:
for i, row_to_compare in tabla_analisis.iterrows():
    for j, row_comparing in ASECORP_BBDD_BOE.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            tabla_analisis['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD_BOE['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

{'Real Decreto 1428/2003'} RD 1428/2003 ESP
{'Real Decreto 1428/2003'} RD 1514/2018 ESP
{'Real Decreto 1428/2003'} RD 667/2015 ESP
{'Real Decreto 1428/2003'} RD 965/2006 ESP
{'Real Decreto Legislativo 6/2015'} RDG 6/2015 ESP


In [1766]:
tabla_analisis

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto,Referencias_completas,Tags,Match_ASECORP_BBDD
0,"[Sistema financiero, Unión Europea]",2021-02-12,"[Orden HAC/114/2021, de 5 de febrero, por la q...",[BOE-A-2021-2045],"[Ayudas, Desarrollo regional, Financiación com...","[BOE-A-2016-12485, BOE-A-2003-20977, DOUE-L-20...","[MODIFICA, DE CONFORMIDAD con, CITA, CITA, CITA]","[las normas 1, 3, 6, 10, 12 a 16, SE AÑADE las...","[MODIFICA las normas 1, 3, 6, 10, 12 a 16, SE ...","[Orden HFP/1979/2016, Ley 38/2003, Reglamento ...",[]
1,[Transportes y tráfico],2021-02-12,"[Resolución de 3 de febrero de 2021, de la Dir...",[BOE-A-2021-2047],"[Carreteras, Circulación vial, Deporte, Mercan...","[BOE-A-2015-11722, BOE-A-2003-23514, BOE-A-201...","[DE CONFORMIDAD con, DE CONFORMIDAD con, EN RE...","[el art. 18 de la Ley sobre Tráfico, Circulaci...",[DE CONFORMIDAD con el art. 18 de la Ley sobre...,"[Real Decreto Legislativo 6/2015, Real Decreto...","[RD 1428/2003 ESP, RD 1514/2018 ESP, RD 667/20..."
2,"[Comercio, Consumidores y usuarios, Sanidad, T...",2021-02-12,"[Orden CSM/115/2021, de 11 de febrero, por la ...",[BOE-A-2021-2046],"[Comercialización, Entidades de acreditación, ...","[BOE-A-2020-4761, BOE-A-2020-4525]","[DE CONFORMIDAD con, EN RELACIÓN con]","[el art. 3 del Real Decreto 495/2020, de 28 de...",[DE CONFORMIDAD con el art. 3 del Real Decreto...,"[Real Decreto 495/2020, Orden SND/354/2020]",[]


In [1767]:
tabla_analisis['Match_ASECORP_BBDD'][1]

['RD 1428/2003 ESP',
 'RD 1514/2018 ESP',
 'RD 667/2015 ESP',
 'RD 965/2006 ESP',
 'RDG 6/2015 ESP']

In [1768]:
tabla_analisis['Tags'][1]

['Real Decreto Legislativo 6/2015',
 'Real Decreto 1428/2003',
 'Resolución de 20 de diciembre de 2019']

# Genera Fichero EXCEL de resultados

In [1769]:
## Cambia orden de columnas y elimina las no necesarias  
tabla_analisis_final = tabla_analisis[['Item_id','Item_Name','Fecha_publicacion','Materias','Alertas','Referencias','Referencias_completas','Tags','Match_ASECORP_BBDD']]

In [1770]:
## Crea función que convierte lista a string en todas las columnas de tabla_analisis
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [1771]:
## Aplica función de conversión de listas a strings
tabla_analisis_final = tabla_analisis_final.apply(lambda x: [list2Str(i) for i in x])

In [1772]:
  tabla_analisis_final[0:10]

,Item_id,Item_Name,Fecha_publicacion,Materias,Alertas,Referencias,Referencias_completas,Tags,Match_ASECORP_BBDD
0,BOE-A-2021-2045,"Orden HAC/114/2021, de 5 de febrero, por la qu...",2021-02-12,"Ayudas, Desarrollo regional, Financiación comu...","Sistema financiero, Unión Europea","BOE-A-2016-12485, BOE-A-2003-20977, DOUE-L-202...","MODIFICA las normas 1, 3, 6, 10, 12 a 16, SE A...","Orden HFP/1979/2016, Ley 38/2003, Reglamento (...",
1,BOE-A-2021-2047,"Resolución de 3 de febrero de 2021, de la Dire...",2021-02-12,"Carreteras, Circulación vial, Deporte, Mercanc...",Transportes y tráfico,"BOE-A-2015-11722, BOE-A-2003-23514, BOE-A-2019...",DE CONFORMIDAD con el art. 18 de la Ley sobre ...,"Real Decreto Legislativo 6/2015, Real Decreto ...","RD 1428/2003 ESP, RD 1514/2018 ESP, RD 667/201..."
2,BOE-A-2021-2046,"Orden CSM/115/2021, de 11 de febrero, por la q...",2021-02-12,"Comercialización, Entidades de acreditación, E...","Comercio, Consumidores y usuarios, Sanidad, Te...","BOE-A-2020-4761, BOE-A-2020-4525",DE CONFORMIDAD con el art. 3 del Real Decreto ...,"Real Decreto 495/2020, Orden SND/354/2020",


In [1773]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

tabla_analisis_final_CSV = tabla_analisis_final

In [1774]:
# Compone y genera enlace a PDF del BOE correspondiente
tabla_analisis_final_CSV['Item_id'] = '=HIPERVINCULO(' + '"https://www.boe.es/boe/dias/' \
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%Y')) + '/'\
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%m')) + '/'\
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%d')) + '/'\
                                                        + 'pdfs/'\
                                                        + tabla_analisis_final_CSV['Item_id'] + '.pdf";'\
                                                        + '"' + tabla_analisis_final_CSV['Item_id'] + '")'

In [1775]:
print(tabla_analisis_final_CSV['Item_id'][0])

=HIPERVINCULO("https://www.boe.es/boe/dias/2021/02/12/pdfs/BOE-A-2021-2045.pdf";"BOE-A-2021-2045")


In [1776]:
# tabla_analisis_final_CSV.to_excel("./ASECORP/Resultados_Matching.xlsx", sheet_name='MATCHING_BOE') 

In [1821]:
tabla_analisis_final_CSV.to_csv("./ASECORP/Resultados_Matching_BOE_" + today.strftime("%Y%m%d") + ".csv", index=False) 

# ------------------------------------------------------------------------

# DOGC

In [1778]:
URL_HTML_resumen =  "https://dogc.gencat.cat/es/index.html?newLang=es_ES&language=es_ES"

In [1779]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get(URL_HTML_resumen)

In [1780]:
r.html.links

{'/ca/index.html?newLang=es_ES&language=es_ES',
 '/ca/pdogc_serveis/cerca-republica/',
 '/ca/pdogc_serveis/pdogc_dogc_mobi/',
 '/es/ajuda/mapaweb/',
 '/es/index.html?newLang=es_ES&language=es_ES',
 '/es/pdogc_base_de_dades_del_dogc/',
 '/es/pdogc_canals_interns/pdogc_cercador_de_normativa/?destParam=cercaAdv',
 '/es/pdogc_canals_interns/pdogc_que_es_el_codi_de_verificacio/',
 '/es/pdogc_eadop/',
 '/es/pdogc_enllacos/',
 '/es/pdogc_preguntes_frequents/',
 '/es/pdogc_serveis/',
 '/es/pdogc_serveis/pdogc_normes_urbanistiques_1985-2002/',
 '/es/pdogc_serveis/pdogc_serveis_de_informacio/',
 '/es/pdogc_serveis/pdogc_serveis_de_informacio/pdogc_alertes_dogc/',
 '/es/pdogc_serveis/pdogc_serveis_de_publicacio_al_dogc/Publicacio_Anuncis_situacio_CoV-2/',
 '/es/pdogc_serveis/pdogc_serveis_de_publicacio_al_dogc/pdogc_dogc_eacat/index.html',
 '/es/pdogc_sobre_el_dogc/',
 'http://llibreria.gencat.cat/index.php?language=es',
 'http://web.gencat.cat/es/temes/',
 'http://www.boe.es/diario_boe/calendari

In [1781]:
r.html.xpath('//*[@id="titol"]/span/text()')

[' DOGC núm.\r\n\t\t\t\t\t 8340 (12/02/2021)']

In [1782]:
r.html.xpath('//*[@id="titol"]/h3/text()')


['\r\n\t\t\t\t\tÚltimo DOGC publicado\r\n\t\t\t\t']

In [1783]:
r.html.lxml.tag

'html'

In [1784]:
r.html.lxml.xpath('//*[@id="titol"]/span')

[<Element span at 0x7fe909ea1220>]

In [1785]:
#import requests
#from lxml import html

response = requests.get(URL_HTML_resumen)

tree = html.fromstring(response.text)

# fetches all useful elements 
useful = tree.xpath('//*[@id="titol"]/span/text()')

print(useful)


[' DOGC núm.\r\n\t\t\t\t\t 8340 (12/02/2021)']


In [1786]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

['/es/pdogc_canals_interns/pdogc_sumari_del_dogc/']

In [1787]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@name')

['anexos']

In [1788]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@value')

['1']

In [1789]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form')

[<Element form at 0x7fe8dde09cc0>]

In [1790]:
useful = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/input[3]/@value')

print(useful)

['8340']


In [1791]:
tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

['/es/pdogc_canals_interns/pdogc_sumari_del_dogc/']

In [1792]:
cadena = ''
for input in tree.xpath('//*[@id="sumari"]/ul/li[1]/form'):
    argumentos = input.xpath('./input/@name')
    valores = input.xpath('./input/@value')
    print(argumentos, valores)

for indice in range(len(argumentos)):
    cadena += argumentos[indice] + '=' + valores[indice] + '&'

argumentos[1]
cadena[:-1]

['anexos', 'language', 'numDOGC', 'seccion'] ['1', 'es_ES', '8340', '0']


'anexos=1&language=es_ES&numDOGC=8340&seccion=0'

## Recoge Nombre Secciones Sumario

In [1793]:
import re
seccion = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/a/text()')
seccion = re.sub('(\\r|\\n|\\t|\ )+', '', seccion[0])
seccion

'DISPOSICIONES'

## Recoge Nombre Secciones Sumario

In [1794]:
secciones = tree.xpath('//*[@id="sumari"]/ul/li')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    seccion = seccion.xpath('./form/a/text()')
    seccion = re.sub('(\\r|\\n|\\t)+', '', seccion[0])
    print(seccion[1:-1])
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion[1:-1]}, ignore_index=True)


   DISPOSICIONES 
   CARGOS Y PERSONAL
   ANUNCIOS DE LA GENERALIDAD DE CATALUÑA
   ANUNCIOS DE LA ADMINISTRACIÓN LOCAL
   ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA 
   ANUNCIOS VARIOS


In [1795]:
df_secciones_sumarios

,Seccion
0,DISPOSICIONES
1,CARGOS Y PERSONAL
2,ANUNCIOS DE LA GENERALIDAD DE CATALUÑA
3,ANUNCIOS DE LA ADMINISTRACIÓN LOCAL
4,ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA
5,ANUNCIOS VARIOS


## Recoge Valores para formar URLs Secciones Sumario

In [1796]:
URL_base_sumario = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

df_URL_sumarios = pd.DataFrame()
for seccion in secciones:    
    cadena = ''
    for input in seccion.xpath('./form'):
        argumentos = input.xpath('./input/@name')
        valores = input.xpath('./input/@value')
        #print(argumentos, valores)

    for indice in range(len(argumentos)):
        cadena += argumentos[indice] + '=' + valores[indice] + '&'

    URL_sumario = 'https://dogc.gencat.cat' + str(URL_base_sumario[0]) + '?' + str(cadena[:-1])
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8340&seccion=0
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8340&seccion=1
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8340&seccion=2
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8340&seccion=3
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8340&seccion=4
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8340&seccion=5


In [1797]:
df_URL_sumarios

,URL_Seccion
0,https://dogc.gencat.cat/es/pdogc_canals_intern...
1,https://dogc.gencat.cat/es/pdogc_canals_intern...
2,https://dogc.gencat.cat/es/pdogc_canals_intern...
3,https://dogc.gencat.cat/es/pdogc_canals_intern...
4,https://dogc.gencat.cat/es/pdogc_canals_intern...
5,https://dogc.gencat.cat/es/pdogc_canals_intern...


In [1798]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [1799]:
df_sumarios

,Seccion,URL_Seccion
0,DISPOSICIONES,https://dogc.gencat.cat/es/pdogc_canals_intern...
1,CARGOS Y PERSONAL,https://dogc.gencat.cat/es/pdogc_canals_intern...
2,ANUNCIOS DE LA GENERALIDAD DE CATALUÑA,https://dogc.gencat.cat/es/pdogc_canals_intern...
3,ANUNCIOS DE LA ADMINISTRACIÓN LOCAL,https://dogc.gencat.cat/es/pdogc_canals_intern...
4,ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA,https://dogc.gencat.cat/es/pdogc_canals_intern...
5,ANUNCIOS VARIOS,https://dogc.gencat.cat/es/pdogc_canals_intern...


In [1800]:
df_sumarios['URL_Seccion'][4]

'https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8340&seccion=4'

## Recoge Items en Seccion Disposiciones

In [1801]:
### Recoge Items en Seccion Disposiciones

response = requests.get(df_sumarios['URL_Seccion'][0])
sumario_HTML = html.fromstring(response.text)

#seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div[1]/text()')
seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div/text()')
bloques = sumario_HTML.xpath('//*[@id="disposicions"]')
df_disposiciones = pd.DataFrame(columns=['Seccion','Item_Name','PDF_link'])
for bloque in bloques: 
    item_name = bloque.xpath('./div/p/text()')
    #item_name = re.sub('(\\r|\\n|\\t)+', '', item_name[0])
    pdf_link = bloque.xpath('./div/div/a[4]/@href')

for row in range(len(pdf_link)):
    df_disposiciones = df_disposiciones.append({'Item_Name': re.sub('(\\r|\\n|\\t)+', '',                                                                               item_name[row]),
                                            'PDF_link' : pdf_link[row],
                                            'Seccion' : seccion[0][:-1]},
                                            ignore_index=True)

df_disposiciones

,Seccion,Item_Name,PDF_link
0,DISPOSICIONES,"RESOLUCIÓN PRE/314/2021, de 9 de febrero, por ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
1,DISPOSICIONES,"RESOLUCIÓN PRE/315/2021, de 9 de febrero, por ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
2,DISPOSICIONES,"RESOLUCIÓN PRE/316/2021, de 9 de febrero, por ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
3,DISPOSICIONES,"RESOLUCIÓN PRE/317/2021, de 9 de febrero, por ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
4,DISPOSICIONES,"RESOLUCIÓN PRE/318/2021, de 9 de febrero, por ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
5,DISPOSICIONES,"RESOLUCIÓN PRE/319/2021, de 9 de febrero, por ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
6,DISPOSICIONES,"RESOLUCIÓN PRE/320/2021, de 9 de febrero, por ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
7,DISPOSICIONES,"RESOLUCIÓN PRE/321/2021, de 9 de febrero, por ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
8,DISPOSICIONES,"RESOLUCIÓN PRE/322/2021, de 9 de febrero, por ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
9,DISPOSICIONES,"RESOLUCIÓN PRE/323/2021, de 9 de febrero, por ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...


In [1802]:
pdf_link

['https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=893241&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=893201&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=893190&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=893168&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=893183&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=893231&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=893234&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=893134&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=893260&type=01&

# DOUE

In [1815]:
# --------------------------------------------------------------------------------------
# DOUE
# --------------------------------------------------------------------------------------

URL_HTML_resumen =  "https://eur-lex.europa.eu/oj/direct-access.html?locale=es"

## carga página HTML y genera árbol

response = requests.get(URL_HTML_resumen)
tree = html.fromstring(response.text)

## Recoge Nombre Secciones Sumario

secciones = tree.xpath('//*[@id="MainContent"]/div[2]/div[2]/div/table/tbody/tr[1]')
#print(secciones)


[]


In [1818]:
source_dir = './DOUEs'
target_dir = './DOUEs_Anteriores'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    #shutil.move(os.path.join(source_dir, file_name), target_dir)
    # Evita que de error si el fichero que se mueve ya existe en dir destino
    try:
        os.remove(os.path.join(target_dir, file_name))
        shutil.move(os.path.join(source_dir, file_name), target_dir)
    except OSError:
        shutil.move(os.path.join(source_dir, file_name), target_dir)

In [1819]:
## Recoge Valores para formar URLs Secciones Sumario

df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    seccion_name = seccion.xpath('./td[2]/a/text()')
    seccion_URL = seccion.xpath('./td[2]/a/@href')
    #seccion = re.sub('(\\r|\\n|\\t)+', '', seccion)
    #print(seccion_name)
    #print(seccion_URL)

for row in range(len(seccion_name)):
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion_name[row],
                                            'Seccion_link' : 'https://eur-lex.europa.eu' + seccion_URL[row][4:]},
                                            ignore_index=True)

In [1820]:
df_secciones_sumarios

,Seccion,Seccion_link
0,L048,https://eur-lex.europa.eu/legal-content/ES/TXT...


In [1807]:
[ print (row) for row in df_secciones_sumarios['Seccion_link']]

https://eur-lex.europa.eu/legal-content/ES/TXT/?uri=OJ:L:2021:048:TOC


[None]

In [1808]:
DOUE_sumarios = pd.DataFrame()

for URL in df_secciones_sumarios['Seccion_link']:
    
    ### Recoge Items en Seccion Legislación

    #print(URL)
    response = requests.get(URL)
    sumario_HTML = html.fromstring(response.text)

    documentos_Name = sumario_HTML.xpath('//*[@class="oj-ti-doc-dur"]/a[1]/text()[1]')
    documentos_URL = sumario_HTML.xpath('//*[@class="oj-ti-doc-dur"]/a[1]/@href')

    for row in range(len(documentos_Name)):
        DOUE_sumarios = DOUE_sumarios.append({'Seccion': 'L' + documentos_URL[row][-7:-4],
                                        'Item_Title': documentos_Name[row],
                                        'Item_Link': 'https://eur-lex.europa.eu' + documentos_URL[row][10:],
                                        #'Item_Tag': documentos_URL[row][-45:-29],
                                        'Item_Doc_Name': '',
                                        'Fecha_publicacion': '',
                                        'PDF_Link': '',
                                        'Tags': '',
                                        'Match_ASECORP_BBDD': ''},
                                        ignore_index=True)


In [1809]:
# documentos_URL[1][-45:-29]

In [1812]:
DOUE_sumarios

""


In [1811]:
print(DOUE_sumarios['Item_Link'][1])

KeyError: 'Item_Link'

In [ ]:
def save_txt(txt, path):
    with open(path, 'w') as f:
        f.write(txt)


In [ ]:
documentos_FileName = sumario_HTML.xpath('//*[@class="DocumentTitle pull-left"]/text()')

In [ ]:
row = 0
for link in DOUE_sumarios['Item_Link']:
    sumario_Text = []
    ### Recoge PDF link en Items Legislación

    response = requests.get(link)
    sumario_HTML = html.fromstring(response.text)

    PDF_Link = sumario_HTML.xpath('//*[@id="format_language_table_PDF_ES"]/@href')
    DOUE_sumarios['PDF_Link'][row] = 'https://eur-lex.europa.eu' + PDF_Link[0][10:]

    Item_Doc_Name = sumario_HTML.xpath('//*[@class="DocumentTitle pull-left"]/text()')[0][9:]
    DOUE_sumarios['Item_Doc_Name'][row] = Item_Doc_Name

    #Item_Fecha = str(sumario_HTML.xpath('//*[@class="oj-hd-date"]/text()'))
    #Item_Fecha = datetime.strptime(str(raiz.xpath('//*[@class="oj-hd-date"]/text()'))[0:-2],"%d.%m.%Y")
    Item_Fecha = re.findall('[0-9]+\.[0-9]+\.[0-9]+', str(sumario_HTML.xpath('//*[@class="oj-hd-date"]/text()')))
    DOUE_sumarios['Fecha_publicacion'][row] = datetime.strptime(str(Item_Fecha)[2:-2],"%d.%m.%Y").date()

    ### Recoge Texto de Items
    
    Item_Text = sumario_HTML.xpath('//*[@class="oj-normal"]/text()')
    
    #[ sumario_Text.append(re.sub('(\\r||\\n\\t|\\xa0)+', '', item)) for item in Item_Text ]
    [ sumario_Text.append(item) for item in Item_Text ]
   
    ### Sustituye caracteres 'n.' por 'n.º' ya que al importar de HTML se pierde el caracter º

    sumario = "".join(sumario_Text).replace(' n.',' n.º')

    ### Busca tags en texto

    DOUE_sumarios['Tags'][row] = list(set(re.findall('|'.join(pattern), sumario, flags=re.IGNORECASE)))

    ### Salva texto de Items en formato TXT

    save_txt(sumario, './DOUEs/Resumen-DOUE-' + Item_Doc_Name + '.txt')
    #print(sumario)
    #print('./DOUEs/Resumen-DOUE-' + Item_Doc_Name + '.txt')
    #print(Item_Doc_Name)

    #print(list(set(re.findall('|'.join(pattern), sumario, flags=re.IGNORECASE))))

    row += 1


In [ ]:
DOUE_sumarios

In [ ]:
### Ordena Columnas
DOUE_sumarios = DOUE_sumarios[['Seccion', 'Item_Doc_Name', 'Fecha_publicacion', 'PDF_Link', 'Item_Title', 'Item_Link', 'Tags', 'Match_ASECORP_BBDD']]
DOUE_sumarios

In [ ]:
### Crea lista plana de Tags sin duplicados
DOUE_flat_list = list(set([item for row in DOUE_sumarios['Tags'] for item in row ]))

In [ ]:
print (DOUE_flat_list)

In [ ]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
list(set(DOUE_flat_list) & set(boletin_ASECORP_flat_list))

In [ ]:
for i, row_to_compare in DOUE_sumarios.iterrows():
    for j, row_comparing in ASECORP_BBDD_BOE.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            DOUE_sumarios['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD_BOE['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

In [ ]:
DOUE_sumarios

In [ ]:
DOUE_sumarios_CSV = pd.DataFrame()
DOUE_sumarios_CSV = DOUE_sumarios

In [ ]:
# Compone y genera enlace a PDF del DOUE correspondiente
DOUE_sumarios_CSV['PDF_Link'] = '=HIPERVINCULO(' + '"' + DOUE_sumarios['PDF_Link'] + '";' + '"' + "Doc: " + DOUE_sumarios['Item_Doc_Name'] + '")'

In [ ]:
DOUE_sumarios_CSV

In [ ]:
DOUE_sumarios_CSV['PDF_Link'][1]

In [1822]:
### Genera fichero CSV

DOUE_sumarios_CSV.to_csv("./ASECORP/Resultados_Matching_DOUE_" + today.strftime("%Y%m%d") + ".csv", index=False) 